In [1]:
#This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, accuracy_score, confusion_matrix, roc_curve
from scipy.stats import zscore, pearsonr, uniform
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, StratifiedKFold, RandomizedSearchCV

from scipy.io import loadmat

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

from sklearn.feature_selection import VarianceThreshold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/widsdatathon2025/SAMPLE_SUBMISSION.xlsx
/kaggle/input/widsdatathon2025/Data Dictionary.xlsx
/kaggle/input/widsdatathon2025/TRAIN_OLD/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv
/kaggle/input/widsdatathon2025/TRAIN_OLD/TRAINING_SOLUTIONS.xlsx
/kaggle/input/widsdatathon2025/TRAIN_OLD/TRAIN_CATEGORICAL_METADATA.xlsx
/kaggle/input/widsdatathon2025/TRAIN_OLD/TRAIN_QUANTITATIVE_METADATA.xlsx
/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx
/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv
/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_CATEGORICAL_METADATA_new.xlsx
/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_QUANTITATIVE_METADATA_new.xlsx
/kaggle/input/widsdatathon2025/TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv
/kaggle/input/widsdatathon2025/TEST/TEST_CATEGORICAL.xlsx
/kaggle/input/widsdatathon2025/TEST/TEST_QUANTITATIVE_METADATA.xlsx


In [2]:
#Caricamento del dataset di train
train_cat = pd.read_excel ("/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_CATEGORICAL_METADATA_new.xlsx")
train_quant = pd.read_excel("/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_QUANTITATIVE_METADATA_new.xlsx")
train_fmri = pd.read_csv("/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv")
train_sol = pd.read_excel("/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx")

#One hot encoding dei dati categorici
for col in train_cat.select_dtypes(include='int').columns:
    train_cat[col] = train_cat[col].astype('category')

columns_to_encode = train_cat.columns[1:].tolist()
train_encoded = pd.get_dummies(train_cat[columns_to_encode], drop_first=True)
train_encoded = train_encoded.applymap(lambda x: 1 if x is True else (0 if x is False else x))

cat_train_final = pd.concat([train_cat.drop(columns=columns_to_encode), train_encoded], axis=1)
cat_train_final.head()


#Merging dei dati categorici (dopo aver fatto l'encoding) con i dati quantitativi e le soluzioni, per ottenere il dataset dei dati demografici
train_demo = pd.merge(cat_train_final, train_quant, on = 'participant_id')
train_final_demo = pd.merge(train_demo, train_sol, on = 'participant_id')
train_final_demo.head()

#Merging dei dati fmri le soluzioni, per ottenere il dataset dei dati di connettività
train_final_connett = pd.merge(train_fmri, train_sol, on = 'participant_id')
train_final_connett.head()


#Caricamento del dataset di test
test_cat = pd.read_excel("/kaggle/input/widsdatathon2025/TEST/TEST_CATEGORICAL.xlsx")
test_quant = pd.read_excel("/kaggle/input/widsdatathon2025/TEST/TEST_QUANTITATIVE_METADATA.xlsx")
test_fmri = pd.read_csv("/kaggle/input/widsdatathon2025/TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv")

#One hot encoding dei dati categorici
for col in test_cat.select_dtypes(include='int').columns:
    test_cat[col] = test_cat[col].astype('category')

columns_to_encode = test_cat.columns[1:].tolist()
test_encoded = pd.get_dummies(test_cat[columns_to_encode], drop_first=True)
test_encoded = test_encoded.applymap(lambda x: 1 if x is True else (0 if x is False else x))

cat_test_final = pd.concat([test_cat.drop(columns=columns_to_encode), test_encoded], axis=1)
cat_test_final.head()


#Merging dei dati categorici (dopo aver fatto l'encoding) con i dati quantitativi, per ottenere il dataset dei dati demografici
test_final_demo = pd.merge(cat_test_final, test_quant, on = 'participant_id')
test_final_demo.head()

#Cambio il nome dei dati fmri per ottenere il dataset dei dati di connettività
test_final_connett = test_fmri


/tmp/ipykernel_13/196046770.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_encoded = train_encoded.applymap(lambda x: 1 if x is True else (0 if x is False else x))
/tmp/ipykernel_13/196046770.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_encoded = test_encoded.applymap(lambda x: 1 if x is True else (0 if x is False else x))


In [3]:
#Controllo quanti valori nulli ci sono nel dataset di train
print(train_final_demo.isna().sum())

#Sostituisco i valori mancanti delle variabili categoriche con la moda
train_final_demo['PreInt_Demos_Fam_Child_Ethnicity'] = train_final_demo['PreInt_Demos_Fam_Child_Ethnicity'].fillna(train_final_demo['PreInt_Demos_Fam_Child_Ethnicity'].mode()[0])
train_final_demo['PreInt_Demos_Fam_Child_Race'] = train_final_demo['PreInt_Demos_Fam_Child_Race'].fillna(train_final_demo['PreInt_Demos_Fam_Child_Race'].mode()[0])
train_final_demo['MRI_Track_Scan_Location'] = train_final_demo['MRI_Track_Scan_Location'].fillna(train_final_demo['MRI_Track_Scan_Location'].mode()[0])
train_final_demo['Barratt_Barratt_P1_Edu'] = train_final_demo['Barratt_Barratt_P1_Edu'].fillna(train_final_demo['Barratt_Barratt_P1_Edu'].mode()[0])
train_final_demo['Barratt_Barratt_P1_Occ'] = train_final_demo['Barratt_Barratt_P1_Occ'].fillna(train_final_demo['Barratt_Barratt_P1_Occ'].mode()[0])
train_final_demo['Barratt_Barratt_P2_Edu'] = train_final_demo['Barratt_Barratt_P2_Edu'].fillna(train_final_demo['Barratt_Barratt_P2_Edu'].mode()[0])
train_final_demo['Barratt_Barratt_P2_Occ'] = train_final_demo['Barratt_Barratt_P2_Occ'].fillna(train_final_demo['Barratt_Barratt_P2_Occ'].mode()[0])

#Sostituisco i valori mancanti delle variabili quantitative con la media
train_final_demo.fillna({'EHQ_EHQ_Total':train_final_demo['EHQ_EHQ_Total'].mean()}, inplace = True)
train_final_demo.fillna({'ColorVision_CV_Score':train_final_demo['ColorVision_CV_Score'].mean()}, inplace = True)
train_final_demo.fillna({'APQ_P_APQ_P_CP':train_final_demo['APQ_P_APQ_P_CP'].mean()}, inplace = True)
train_final_demo.fillna({'APQ_P_APQ_P_ID':train_final_demo['APQ_P_APQ_P_ID'].mean()}, inplace = True)
train_final_demo.fillna({'APQ_P_APQ_P_INV':train_final_demo['APQ_P_APQ_P_INV'].mean()}, inplace = True)
train_final_demo.fillna({'APQ_P_APQ_P_OPD':train_final_demo['APQ_P_APQ_P_OPD'].mean()}, inplace = True)
train_final_demo.fillna({'APQ_P_APQ_P_PM':train_final_demo['APQ_P_APQ_P_PM'].mean()}, inplace = True)
train_final_demo.fillna({'APQ_P_APQ_P_PP':train_final_demo['APQ_P_APQ_P_PP'].mean()}, inplace = True)
train_final_demo.fillna({'SDQ_SDQ_Conduct_Problems':train_final_demo['SDQ_SDQ_Conduct_Problems'].mean()}, inplace = True)
train_final_demo.fillna({'SDQ_SDQ_Difficulties_Total':train_final_demo['SDQ_SDQ_Difficulties_Total'].mean()}, inplace = True)
train_final_demo.fillna({'SDQ_SDQ_Emotional_Problems':train_final_demo['SDQ_SDQ_Emotional_Problems'].mean()}, inplace = True)
train_final_demo.fillna({'SDQ_SDQ_Externalizing':train_final_demo['SDQ_SDQ_Externalizing'].mean()}, inplace = True)
train_final_demo.fillna({'SDQ_SDQ_Generating_Impact':train_final_demo['SDQ_SDQ_Generating_Impact'].mean()}, inplace = True)
train_final_demo.fillna({'SDQ_SDQ_Hyperactivity':train_final_demo['SDQ_SDQ_Hyperactivity'].mean()}, inplace = True)
train_final_demo.fillna({'SDQ_SDQ_Internalizing':train_final_demo['SDQ_SDQ_Internalizing'].mean()}, inplace = True)
train_final_demo.fillna({'SDQ_SDQ_Peer_Problems':train_final_demo['SDQ_SDQ_Peer_Problems'].mean()}, inplace = True)
train_final_demo.fillna({'SDQ_SDQ_Prosocial':train_final_demo['SDQ_SDQ_Prosocial'].mean()}, inplace = True)
train_final_demo.fillna({'MRI_Track_Age_at_Scan':train_final_demo['MRI_Track_Age_at_Scan'].mean()}, inplace = True)

#Ora non ci sono più NaN values
print('\n\n')
print(train_final_demo.isna().sum())
print('\n')
print(train_final_demo.isna().sum().sum())


#Controllo quanti valori nulli ci sono nel dataset di test
print(test_final_demo.isna().sum())

#Sostituisco i valori mancanti delle variabili categoriche con la moda
test_final_demo['PreInt_Demos_Fam_Child_Ethnicity'] = test_final_demo['PreInt_Demos_Fam_Child_Ethnicity'].fillna(test_final_demo['PreInt_Demos_Fam_Child_Ethnicity'].mode()[0])
test_final_demo['PreInt_Demos_Fam_Child_Race'] = test_final_demo['PreInt_Demos_Fam_Child_Race'].fillna(test_final_demo['PreInt_Demos_Fam_Child_Race'].mode()[0])
test_final_demo['Barratt_Barratt_P1_Edu'] = test_final_demo['Barratt_Barratt_P1_Edu'].fillna(test_final_demo['Barratt_Barratt_P1_Edu'].mode()[0])
test_final_demo['Barratt_Barratt_P1_Occ'] = test_final_demo['Barratt_Barratt_P1_Occ'].fillna(test_final_demo['Barratt_Barratt_P1_Occ'].mode()[0])
test_final_demo['Barratt_Barratt_P2_Edu'] = test_final_demo['Barratt_Barratt_P2_Edu'].fillna(test_final_demo['Barratt_Barratt_P2_Edu'].mode()[0])
test_final_demo['Barratt_Barratt_P2_Occ'] = test_final_demo['Barratt_Barratt_P2_Occ'].fillna(test_final_demo['Barratt_Barratt_P2_Occ'].mode()[0])

# Sostituisco i valori mancanti delle variabili quantitative con la media
test_final_demo.fillna({'EHQ_EHQ_Total': test_final_demo['EHQ_EHQ_Total'].mean()}, inplace=True)
test_final_demo.fillna({'ColorVision_CV_Score': test_final_demo['ColorVision_CV_Score'].mean()}, inplace=True)
test_final_demo.fillna({'APQ_P_APQ_P_CP': test_final_demo['APQ_P_APQ_P_CP'].mean()}, inplace=True)
test_final_demo.fillna({'APQ_P_APQ_P_ID': test_final_demo['APQ_P_APQ_P_ID'].mean()}, inplace=True)
test_final_demo.fillna({'APQ_P_APQ_P_INV': test_final_demo['APQ_P_APQ_P_INV'].mean()}, inplace=True)
test_final_demo.fillna({'APQ_P_APQ_P_OPD': test_final_demo['APQ_P_APQ_P_OPD'].mean()}, inplace=True)
test_final_demo.fillna({'APQ_P_APQ_P_PM': test_final_demo['APQ_P_APQ_P_PM'].mean()}, inplace=True)
test_final_demo.fillna({'APQ_P_APQ_P_PP': test_final_demo['APQ_P_APQ_P_PP'].mean()}, inplace=True)
test_final_demo.fillna({'SDQ_SDQ_Conduct_Problems': test_final_demo['SDQ_SDQ_Conduct_Problems'].mean()}, inplace=True)
test_final_demo.fillna({'SDQ_SDQ_Difficulties_Total': test_final_demo['SDQ_SDQ_Difficulties_Total'].mean()}, inplace=True)
test_final_demo.fillna({'SDQ_SDQ_Emotional_Problems': test_final_demo['SDQ_SDQ_Emotional_Problems'].mean()}, inplace=True)
test_final_demo.fillna({'SDQ_SDQ_Externalizing': test_final_demo['SDQ_SDQ_Externalizing'].mean()}, inplace=True)
test_final_demo.fillna({'SDQ_SDQ_Generating_Impact': test_final_demo['SDQ_SDQ_Generating_Impact'].mean()}, inplace=True)
test_final_demo.fillna({'SDQ_SDQ_Hyperactivity': test_final_demo['SDQ_SDQ_Hyperactivity'].mean()}, inplace=True)
test_final_demo.fillna({'SDQ_SDQ_Internalizing': test_final_demo['SDQ_SDQ_Internalizing'].mean()}, inplace=True)
test_final_demo.fillna({'SDQ_SDQ_Peer_Problems': test_final_demo['SDQ_SDQ_Peer_Problems'].mean()}, inplace=True)
test_final_demo.fillna({'SDQ_SDQ_Prosocial': test_final_demo['SDQ_SDQ_Prosocial'].mean()}, inplace=True)
test_final_demo.fillna({'MRI_Track_Age_at_Scan': test_final_demo['MRI_Track_Age_at_Scan'].mean()}, inplace=True)


#Ora non ci sono più NaN values
print('\n\n')
print(test_final_demo.isna().sum())
print('\n')
print(test_final_demo.isna().sum().sum())

participant_id                        0
PreInt_Demos_Fam_Child_Ethnicity     43
PreInt_Demos_Fam_Child_Race          54
MRI_Track_Scan_Location               3
Barratt_Barratt_P1_Edu               15
Barratt_Barratt_P1_Occ               31
Barratt_Barratt_P2_Edu              198
Barratt_Barratt_P2_Occ              222
Basic_Demos_Enroll_Year_2016          0
Basic_Demos_Enroll_Year_2017          0
Basic_Demos_Enroll_Year_2018          0
Basic_Demos_Enroll_Year_2019          0
Basic_Demos_Enroll_Year_2020          0
Basic_Demos_Study_Site_2              0
Basic_Demos_Study_Site_3              0
Basic_Demos_Study_Site_4              0
EHQ_EHQ_Total                        13
ColorVision_CV_Score                 23
APQ_P_APQ_P_CP                       12
APQ_P_APQ_P_ID                       12
APQ_P_APQ_P_INV                      12
APQ_P_APQ_P_OPD                      12
APQ_P_APQ_P_PM                       12
APQ_P_APQ_P_PP                       12
SDQ_SDQ_Conduct_Problems              9


In [4]:
# === FEATURE SELECTION MANUALE PER DATI DI CONNETTIVITÀ ===
#DATI DI TRAIN

# Numeri interi che vuoi cercare (come stringhe)
numeri_da_cercare_str = ['76', '185', '75', '184', '80', '81', '82', '187', '188', '189', '77', '78', '79', '186', '191', '88', '89', '90', '91', '192', '87', '83', '84', '85', '86', '190', '92', '93', '94', '95', '193', '96', '194', '98', '196', '97', '195', '61', '167', '58', '59', '60', '165', '166', '63', '64', '65', '168', '169', '62', '66', '170', '57', '68', '173', '174', '175', '176', '180', '69', '70', '71', '177', '178', '179', '67', '171', '172', '74', '183', '72', '73', '181', '182', '31', '132', '32', '33', '34', '133', '134', '135', '136', '29', '30', '131', '39', '141', '35', '36', '37', '38', '137', '138', '139', '140']

# Lista per memorizzare i nomi delle colonne da mantenere
colonne_da_mantenere = ['participant_id', 'Sex_F', 'ADHD_Outcome']

# Itera sui nomi delle colonne di connettività
colonne_connettivita = [col for col in train_final_connett.columns if col not in ['participant_id', 'Sex_F', 'ADHD_Outcome']]

for col in colonne_connettivita:
    for numero_str in numeri_da_cercare_str:
        # Cerchiamo il numero come sottostringa intera, evitando '10', '22', ecc.
        if numero_str in col:
            try:
                # Tentiamo di estrarre la parte prima e dopo il numero trovato
                index = col.find(numero_str)
                parte_prima = col[:index]
                parte_dopo = col[index + len(numero_str):]

                # Verifichiamo che il numero trovato sia "intero", ovvero delimitato da non-cifre o inizio/fine stringa
                if (not parte_prima or not parte_prima[-1].isdigit()) and (not parte_dopo or not parte_dopo[0].isdigit()):
                    colonne_da_mantenere.append(col)
                    break # Se troviamo il numero intero, manteniamo la colonna e passiamo alla successiva
            except ValueError:
                pass

# Rimuovi i duplicati
colonne_da_mantenere = list(set(colonne_da_mantenere))

# Crea il DataFrame filtrato
train_connett_manual = train_final_connett[colonne_da_mantenere]
train_final_connett = train_connett_manual

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

# ========== PRIMA: MANUALE + AUTOMATICA ==========

# Manuale
numeri_da_cercare_str = ['76', '185', '75', '184', '80', '81', '82', '187', '188', '189', '77', '78', '79', '186', '191', '88', '89', '90', '91', '192', '87', '83', '84', '85', '86', '190', '92', '93', '94', '95', '193', '96', '194', '98', '196', '97', '195', '61', '167', '58', '59', '60', '165', '166', '63', '64', '65', '168', '169', '62', '66', '170', '57', '68', '173', '174', '175', '176', '180', '69', '70', '71', '177', '178', '179', '67', '171', '172', '74', '183', '72', '73', '181', '182', '31', '132', '32', '33', '34', '133', '134', '135', '136', '29', '30', '131', '39', '141', '35', '36', '37', '38', '137', '138', '139', '140']
colonne_da_mantenere = ['participant_id', 'Sex_F', 'ADHD_Outcome']
colonne_connettivita = [col for col in train_final_connett.columns if col not in colonne_da_mantenere]

for col in colonne_connettivita:
    for numero_str in numeri_da_cercare_str:
        if numero_str in col:
            index = col.find(numero_str)
            parte_prima = col[:index]
            parte_dopo = col[index + len(numero_str):]
            if (not parte_prima or not parte_prima[-1].isdigit()) and (not parte_dopo or not parte_dopo[0].isdigit()):
                colonne_da_mantenere.append(col)
                break

colonne_da_mantenere = list(set(colonne_da_mantenere))
train_connett_manual = train_final_connett[colonne_da_mantenere]

# Automatica
y_sex = train_connett_manual['Sex_F'].astype(int)
X_sex = train_connett_manual.drop(columns=['participant_id', 'Sex_F'])

selector = SelectKBest(score_func=f_classif, k=4000)
X_selected = selector.fit_transform(X_sex, y_sex)
selected_features = X_sex.columns[selector.get_support()]

train_final_connett_auto = pd.concat(
    [train_connett_manual[['participant_id', 'Sex_F', 'ADHD_Outcome']], X_sex[selected_features]],
    axis=1
)

# Modello SVM - FS AUTOMATICA + MANUALE
X = train_final_connett_auto.drop(columns=['participant_id', 'Sex_F'])
y = train_final_connett_auto['Sex_F'].astype(int)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
f1_scores_sex_auto = []

for train_idx, test_idx in skf.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = SVC(kernel='linear', class_weight='balanced', random_state=42)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)

    f1_scores_sex_auto.append(f1_score(y_test, y_pred))


# ========== ORA: SOLO MANUALE ==========

train_final_connett_manu = train_connett_manual.copy()  # già filtrato manualmente

# Modello SVM - SOLO FS MANUALE
X = train_final_connett_manu.drop(columns=['participant_id', 'Sex_F'])
y = train_final_connett_manu['Sex_F'].astype(int)

f1_scores_sex_manu = []

for train_idx, test_idx in skf.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = SVC(kernel='linear', class_weight='balanced', random_state=42)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)

    f1_scores_sex_manu.append(f1_score(y_test, y_pred))


# ========== METRICHE ==========

print("\n=== RISULTATI F1-SCORE SESSO ===")
print("📊 FS MANUALE + AUTOMATICA:")
print(f"   ➤ F1-score medio: {np.mean(f1_scores_sex_auto):.4f}")
print(f"   ➤ Dev. standard: {np.std(f1_scores_sex_auto):.4f}")

print("\n🛠 FS SOLO MANUALE:")
print(f"   ➤ F1-score medio: {np.mean(f1_scores_sex_manu):.4f}")
print(f"   ➤ Dev. standard: {np.std(f1_scores_sex_manu):.4f}")


=== RISULTATI F1-SCORE SESSO ===
📊 FS MANUALE + AUTOMATICA:
   ➤ F1-score medio: 0.6810
   ➤ Dev. standard: 0.0482

🛠 FS SOLO MANUALE:
   ➤ F1-score medio: 0.5883
   ➤ Dev. standard: 0.0697


In [6]:
f1_scores_sex_manu = pd.DataFrame(f1_scores_sex_manu, columns=["F1_sex_score"])
f1_scores_sex_manu.to_csv('tesi_4.4_SVC_sex_manuale.csv', index=False)
print("File 'tesi_4.4_SVC_sex_manuale.csv' salvato con successo!")
f1_scores_sex_manu.head()

f1_scores_sex_auto = pd.DataFrame(f1_scores_sex_auto, columns=["F1_sex_score"])
f1_scores_sex_auto.to_csv('tesi_4.4_SVC_sex_mista.csv', index=False)
print("File 'tesi_4.4_SVC_sex_mista.csv' salvato con successo!")
f1_scores_sex_auto.head()


import os
print(os.listdir('.')) # Mostra i file nella directory corrente

File 'tesi_4.4_SVC_sex_manuale.csv' salvato con successo!
File 'tesi_4.4_SVC_sex_mista.csv' salvato con successo!
['tesi_4.4_SVC_sex_mista.csv', '__notebook__.ipynb', 'tesi_4.4_SVC_sex_manuale.csv']


In [7]:
#This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, accuracy_score, confusion_matrix, roc_curve
from scipy.stats import zscore, pearsonr, uniform
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, StratifiedKFold, RandomizedSearchCV

from scipy.io import loadmat

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

from sklearn.feature_selection import VarianceThreshold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/widsdatathon2025/SAMPLE_SUBMISSION.xlsx
/kaggle/input/widsdatathon2025/Data Dictionary.xlsx
/kaggle/input/widsdatathon2025/TRAIN_OLD/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv
/kaggle/input/widsdatathon2025/TRAIN_OLD/TRAINING_SOLUTIONS.xlsx
/kaggle/input/widsdatathon2025/TRAIN_OLD/TRAIN_CATEGORICAL_METADATA.xlsx
/kaggle/input/widsdatathon2025/TRAIN_OLD/TRAIN_QUANTITATIVE_METADATA.xlsx
/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx
/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv
/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_CATEGORICAL_METADATA_new.xlsx
/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_QUANTITATIVE_METADATA_new.xlsx
/kaggle/input/widsdatathon2025/TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv
/kaggle/input/widsdatathon2025/TEST/TEST_CATEGORICAL.xlsx
/kaggle/input/widsdatathon2025/TEST/TEST_QUANTITATIVE_METADATA.xlsx


In [8]:
#Caricamento del dataset di train
train_cat = pd.read_excel ("/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_CATEGORICAL_METADATA_new.xlsx")
train_quant = pd.read_excel("/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_QUANTITATIVE_METADATA_new.xlsx")
train_fmri = pd.read_csv("/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv")
train_sol = pd.read_excel("/kaggle/input/widsdatathon2025/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx")

#One hot encoding dei dati categorici
for col in train_cat.select_dtypes(include='int').columns:
    train_cat[col] = train_cat[col].astype('category')

columns_to_encode = train_cat.columns[1:].tolist()
train_encoded = pd.get_dummies(train_cat[columns_to_encode], drop_first=True)
train_encoded = train_encoded.applymap(lambda x: 1 if x is True else (0 if x is False else x))

cat_train_final = pd.concat([train_cat.drop(columns=columns_to_encode), train_encoded], axis=1)
cat_train_final.head()


#Merging dei dati categorici (dopo aver fatto l'encoding) con i dati quantitativi e le soluzioni, per ottenere il dataset dei dati demografici
train_demo = pd.merge(cat_train_final, train_quant, on = 'participant_id')
train_final_demo = pd.merge(train_demo, train_sol, on = 'participant_id')
train_final_demo.head()

#Merging dei dati fmri le soluzioni, per ottenere il dataset dei dati di connettività
train_final_connett = pd.merge(train_fmri, train_sol, on = 'participant_id')
train_final_connett.head()


#Caricamento del dataset di test
test_cat = pd.read_excel("/kaggle/input/widsdatathon2025/TEST/TEST_CATEGORICAL.xlsx")
test_quant = pd.read_excel("/kaggle/input/widsdatathon2025/TEST/TEST_QUANTITATIVE_METADATA.xlsx")
test_fmri = pd.read_csv("/kaggle/input/widsdatathon2025/TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv")

#One hot encoding dei dati categorici
for col in test_cat.select_dtypes(include='int').columns:
    test_cat[col] = test_cat[col].astype('category')

columns_to_encode = test_cat.columns[1:].tolist()
test_encoded = pd.get_dummies(test_cat[columns_to_encode], drop_first=True)
test_encoded = test_encoded.applymap(lambda x: 1 if x is True else (0 if x is False else x))

cat_test_final = pd.concat([test_cat.drop(columns=columns_to_encode), test_encoded], axis=1)
cat_test_final.head()


#Merging dei dati categorici (dopo aver fatto l'encoding) con i dati quantitativi, per ottenere il dataset dei dati demografici
test_final_demo = pd.merge(cat_test_final, test_quant, on = 'participant_id')
test_final_demo.head()

#Cambio il nome dei dati fmri per ottenere il dataset dei dati di connettività
test_final_connett = test_fmri

/tmp/ipykernel_13/739009659.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_encoded = train_encoded.applymap(lambda x: 1 if x is True else (0 if x is False else x))
/tmp/ipykernel_13/739009659.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_encoded = test_encoded.applymap(lambda x: 1 if x is True else (0 if x is False else x))


In [9]:
#Controllo quanti valori nulli ci sono nel dataset di train
print(train_final_demo.isna().sum())

#Sostituisco i valori mancanti delle variabili categoriche con la moda
train_final_demo['PreInt_Demos_Fam_Child_Ethnicity'] = train_final_demo['PreInt_Demos_Fam_Child_Ethnicity'].fillna(train_final_demo['PreInt_Demos_Fam_Child_Ethnicity'].mode()[0])
train_final_demo['PreInt_Demos_Fam_Child_Race'] = train_final_demo['PreInt_Demos_Fam_Child_Race'].fillna(train_final_demo['PreInt_Demos_Fam_Child_Race'].mode()[0])
train_final_demo['MRI_Track_Scan_Location'] = train_final_demo['MRI_Track_Scan_Location'].fillna(train_final_demo['MRI_Track_Scan_Location'].mode()[0])
train_final_demo['Barratt_Barratt_P1_Edu'] = train_final_demo['Barratt_Barratt_P1_Edu'].fillna(train_final_demo['Barratt_Barratt_P1_Edu'].mode()[0])
train_final_demo['Barratt_Barratt_P1_Occ'] = train_final_demo['Barratt_Barratt_P1_Occ'].fillna(train_final_demo['Barratt_Barratt_P1_Occ'].mode()[0])
train_final_demo['Barratt_Barratt_P2_Edu'] = train_final_demo['Barratt_Barratt_P2_Edu'].fillna(train_final_demo['Barratt_Barratt_P2_Edu'].mode()[0])
train_final_demo['Barratt_Barratt_P2_Occ'] = train_final_demo['Barratt_Barratt_P2_Occ'].fillna(train_final_demo['Barratt_Barratt_P2_Occ'].mode()[0])

#Sostituisco i valori mancanti delle variabili quantitative con la media
train_final_demo.fillna({'EHQ_EHQ_Total':train_final_demo['EHQ_EHQ_Total'].mean()}, inplace = True)
train_final_demo.fillna({'ColorVision_CV_Score':train_final_demo['ColorVision_CV_Score'].mean()}, inplace = True)
train_final_demo.fillna({'APQ_P_APQ_P_CP':train_final_demo['APQ_P_APQ_P_CP'].mean()}, inplace = True)
train_final_demo.fillna({'APQ_P_APQ_P_ID':train_final_demo['APQ_P_APQ_P_ID'].mean()}, inplace = True)
train_final_demo.fillna({'APQ_P_APQ_P_INV':train_final_demo['APQ_P_APQ_P_INV'].mean()}, inplace = True)
train_final_demo.fillna({'APQ_P_APQ_P_OPD':train_final_demo['APQ_P_APQ_P_OPD'].mean()}, inplace = True)
train_final_demo.fillna({'APQ_P_APQ_P_PM':train_final_demo['APQ_P_APQ_P_PM'].mean()}, inplace = True)
train_final_demo.fillna({'APQ_P_APQ_P_PP':train_final_demo['APQ_P_APQ_P_PP'].mean()}, inplace = True)
train_final_demo.fillna({'SDQ_SDQ_Conduct_Problems':train_final_demo['SDQ_SDQ_Conduct_Problems'].mean()}, inplace = True)
train_final_demo.fillna({'SDQ_SDQ_Difficulties_Total':train_final_demo['SDQ_SDQ_Difficulties_Total'].mean()}, inplace = True)
train_final_demo.fillna({'SDQ_SDQ_Emotional_Problems':train_final_demo['SDQ_SDQ_Emotional_Problems'].mean()}, inplace = True)
train_final_demo.fillna({'SDQ_SDQ_Externalizing':train_final_demo['SDQ_SDQ_Externalizing'].mean()}, inplace = True)
train_final_demo.fillna({'SDQ_SDQ_Generating_Impact':train_final_demo['SDQ_SDQ_Generating_Impact'].mean()}, inplace = True)
train_final_demo.fillna({'SDQ_SDQ_Hyperactivity':train_final_demo['SDQ_SDQ_Hyperactivity'].mean()}, inplace = True)
train_final_demo.fillna({'SDQ_SDQ_Internalizing':train_final_demo['SDQ_SDQ_Internalizing'].mean()}, inplace = True)
train_final_demo.fillna({'SDQ_SDQ_Peer_Problems':train_final_demo['SDQ_SDQ_Peer_Problems'].mean()}, inplace = True)
train_final_demo.fillna({'SDQ_SDQ_Prosocial':train_final_demo['SDQ_SDQ_Prosocial'].mean()}, inplace = True)
train_final_demo.fillna({'MRI_Track_Age_at_Scan':train_final_demo['MRI_Track_Age_at_Scan'].mean()}, inplace = True)

#Ora non ci sono più NaN values
print('\n\n')
print(train_final_demo.isna().sum())
print('\n')
print(train_final_demo.isna().sum().sum())


#Controllo quanti valori nulli ci sono nel dataset di test
print(test_final_demo.isna().sum())

#Sostituisco i valori mancanti delle variabili categoriche con la moda
test_final_demo['PreInt_Demos_Fam_Child_Ethnicity'] = test_final_demo['PreInt_Demos_Fam_Child_Ethnicity'].fillna(test_final_demo['PreInt_Demos_Fam_Child_Ethnicity'].mode()[0])
test_final_demo['PreInt_Demos_Fam_Child_Race'] = test_final_demo['PreInt_Demos_Fam_Child_Race'].fillna(test_final_demo['PreInt_Demos_Fam_Child_Race'].mode()[0])
test_final_demo['Barratt_Barratt_P1_Edu'] = test_final_demo['Barratt_Barratt_P1_Edu'].fillna(test_final_demo['Barratt_Barratt_P1_Edu'].mode()[0])
test_final_demo['Barratt_Barratt_P1_Occ'] = test_final_demo['Barratt_Barratt_P1_Occ'].fillna(test_final_demo['Barratt_Barratt_P1_Occ'].mode()[0])
test_final_demo['Barratt_Barratt_P2_Edu'] = test_final_demo['Barratt_Barratt_P2_Edu'].fillna(test_final_demo['Barratt_Barratt_P2_Edu'].mode()[0])
test_final_demo['Barratt_Barratt_P2_Occ'] = test_final_demo['Barratt_Barratt_P2_Occ'].fillna(test_final_demo['Barratt_Barratt_P2_Occ'].mode()[0])

# Sostituisco i valori mancanti delle variabili quantitative con la media
test_final_demo.fillna({'EHQ_EHQ_Total': test_final_demo['EHQ_EHQ_Total'].mean()}, inplace=True)
test_final_demo.fillna({'ColorVision_CV_Score': test_final_demo['ColorVision_CV_Score'].mean()}, inplace=True)
test_final_demo.fillna({'APQ_P_APQ_P_CP': test_final_demo['APQ_P_APQ_P_CP'].mean()}, inplace=True)
test_final_demo.fillna({'APQ_P_APQ_P_ID': test_final_demo['APQ_P_APQ_P_ID'].mean()}, inplace=True)
test_final_demo.fillna({'APQ_P_APQ_P_INV': test_final_demo['APQ_P_APQ_P_INV'].mean()}, inplace=True)
test_final_demo.fillna({'APQ_P_APQ_P_OPD': test_final_demo['APQ_P_APQ_P_OPD'].mean()}, inplace=True)
test_final_demo.fillna({'APQ_P_APQ_P_PM': test_final_demo['APQ_P_APQ_P_PM'].mean()}, inplace=True)
test_final_demo.fillna({'APQ_P_APQ_P_PP': test_final_demo['APQ_P_APQ_P_PP'].mean()}, inplace=True)
test_final_demo.fillna({'SDQ_SDQ_Conduct_Problems': test_final_demo['SDQ_SDQ_Conduct_Problems'].mean()}, inplace=True)
test_final_demo.fillna({'SDQ_SDQ_Difficulties_Total': test_final_demo['SDQ_SDQ_Difficulties_Total'].mean()}, inplace=True)
test_final_demo.fillna({'SDQ_SDQ_Emotional_Problems': test_final_demo['SDQ_SDQ_Emotional_Problems'].mean()}, inplace=True)
test_final_demo.fillna({'SDQ_SDQ_Externalizing': test_final_demo['SDQ_SDQ_Externalizing'].mean()}, inplace=True)
test_final_demo.fillna({'SDQ_SDQ_Generating_Impact': test_final_demo['SDQ_SDQ_Generating_Impact'].mean()}, inplace=True)
test_final_demo.fillna({'SDQ_SDQ_Hyperactivity': test_final_demo['SDQ_SDQ_Hyperactivity'].mean()}, inplace=True)
test_final_demo.fillna({'SDQ_SDQ_Internalizing': test_final_demo['SDQ_SDQ_Internalizing'].mean()}, inplace=True)
test_final_demo.fillna({'SDQ_SDQ_Peer_Problems': test_final_demo['SDQ_SDQ_Peer_Problems'].mean()}, inplace=True)
test_final_demo.fillna({'SDQ_SDQ_Prosocial': test_final_demo['SDQ_SDQ_Prosocial'].mean()}, inplace=True)
test_final_demo.fillna({'MRI_Track_Age_at_Scan': test_final_demo['MRI_Track_Age_at_Scan'].mean()}, inplace=True)


#Ora non ci sono più NaN values
print('\n\n')
print(test_final_demo.isna().sum())
print('\n')
print(test_final_demo.isna().sum().sum())

participant_id                        0
PreInt_Demos_Fam_Child_Ethnicity     43
PreInt_Demos_Fam_Child_Race          54
MRI_Track_Scan_Location               3
Barratt_Barratt_P1_Edu               15
Barratt_Barratt_P1_Occ               31
Barratt_Barratt_P2_Edu              198
Barratt_Barratt_P2_Occ              222
Basic_Demos_Enroll_Year_2016          0
Basic_Demos_Enroll_Year_2017          0
Basic_Demos_Enroll_Year_2018          0
Basic_Demos_Enroll_Year_2019          0
Basic_Demos_Enroll_Year_2020          0
Basic_Demos_Study_Site_2              0
Basic_Demos_Study_Site_3              0
Basic_Demos_Study_Site_4              0
EHQ_EHQ_Total                        13
ColorVision_CV_Score                 23
APQ_P_APQ_P_CP                       12
APQ_P_APQ_P_ID                       12
APQ_P_APQ_P_INV                      12
APQ_P_APQ_P_OPD                      12
APQ_P_APQ_P_PM                       12
APQ_P_APQ_P_PP                       12
SDQ_SDQ_Conduct_Problems              9


In [10]:
# === FEATURE SELECTION MANUALE PER DATI DEMOGRAFICI ===
# DATI DI TRAIN
from sklearn.feature_selection import SelectKBest, f_classif


# Lista delle features (colonne) che vuoi eliminare
features_da_eliminare = ["Basic_Demos_Study_Site_2", "Basic_Demos_Study_Site_3", "Basic_Demos_Study_Site_4", "MRI_Track_Scan_Location"]

# Salva numero di feature iniziali (INCLUDENDO ID e target)
n_feature_iniziali = train_final_demo.shape[1]

# Elimina le features specificate dal DataFrame
train_final_demo = train_final_demo.drop(columns=features_da_eliminare, errors='ignore')

# Salva numero di feature dopo rimozione manuale
n_feature_post_manuale = train_final_demo.shape[1]



# === FEATURE SELECTION AUTOMATICA PER DATI DEMOGRAFICI (SelectKBest) ===
# DATI DI TRAIN

# Colonne da mantenere sempre nel dataset finale
colonne_da_aggiungere = ['participant_id','ADHD_Outcome']
colonne_da_aggiungere_presenti = [col for col in colonne_da_aggiungere if col in train_final_demo.columns]

# Seleziona solo le colonne numeriche (int o float)
train_numeriche = train_final_demo.select_dtypes(include=['int64', 'float64'])

# Escludi le colonne da mantenere
feature_data = train_numeriche.drop(columns=[col for col in colonne_da_aggiungere_presenti if col in train_numeriche.columns], errors='ignore')

# Estrai il target per la selezione automatica
y_kbest = train_final_demo['ADHD_Outcome'].astype(int)

# Applica SelectKBest
k = 26
selector = SelectKBest(score_func=f_classif, k=k)
X_selected = selector.fit_transform(feature_data, y_kbest)

# Ottieni i nomi delle feature selezionate
feature_mask = selector.get_support()
selected_features = feature_data.columns[feature_mask]

# Costruisci il nuovo DataFrame
train_final_demo_selected = pd.concat(
    [train_final_demo[colonne_da_aggiungere_presenti], feature_data[selected_features]],
    axis=1
)

# Output finale
print(f"🔢 Numero di feature iniziali (INCLUSI ID e target): {n_feature_iniziali}")
print(f"🧹 Numero di feature dopo la selezione manuale: {n_feature_post_manuale}")
print(f"🔍 Numero di feature selezionate automaticamente (SelectKBest): {len(selected_features)}")
print(f"⚙️ Numero totale di colonne finali (con ID e target): {train_final_demo_selected.shape[1]}")
print("\n📌 Feature selezionate:", list(selected_features))


# === MODELLAZIONE CON RANDOM FOREST (300 estimatori) PER ADHD USANDO DATI DEMOGRAFICI ===

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Features e target
X = train_final_demo_selected.drop(columns=['participant_id', 'ADHD_Outcome'])
y = train_final_demo_selected['ADHD_Outcome'].astype(int)

# Cross-validation stratificata su ADHD_Outcome
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Liste per salvare i risultati di ogni fold
acc_scores_adhd = []
f1_scores_adhd = []

# Cross-validation
for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
    print(f"\n=== Fold {fold} ===")

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    print(f"  Dimensione X_train: {X_train.shape}")
    print(f"  Dimensione X_test: {X_test.shape}")
    print(f"  Dimensione y_train: {y_train.shape}")
    print(f"  Dimensione y_test: {y_test.shape}")
    print(f"  Valori unici in y_train: {np.unique(y_train)}")
    print(f"  Valori unici in y_test: {np.unique(y_test)}")

    # === Standardizzazione === (facoltativa per Random Forest, ma la manteniamo per coerenza)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # === Modello Random Forest ===
    rf = RandomForestClassifier(
        n_estimators=300,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    )
    rf.fit(X_train_scaled, y_train)

    y_pred = rf.predict(X_test_scaled)

    print(f"  Predizioni y_pred (prime 5): {y_pred[:5]}")
    print(f"  Valori reali y_test (prime 5): {y_test[:5].values}")

    # Metriche per questo fold
    acc_scores_adhd.append(accuracy_score(y_test, y_pred))
    f1_scores_adhd.append(f1_score(y_test, y_pred))


# === RISULTATI FINALI ===
print("\n=== Media delle metriche su 10 fold ===")
print(f"F1-score medio ADHD_Outcome: {np.mean(f1_scores_adhd):.4f}")
print(f"Deviazione standard F1-score ADHD_Outcome: {np.std(f1_scores_adhd):.4f}")




🔢 Numero di feature iniziali (INCLUSI ID e target): 36
🧹 Numero di feature dopo la selezione manuale: 32
🔍 Numero di feature selezionate automaticamente (SelectKBest): 26
⚙️ Numero totale di colonne finali (con ID e target): 28

📌 Feature selezionate: ['PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race', 'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Occ', 'Basic_Demos_Enroll_Year_2016', 'Basic_Demos_Enroll_Year_2017', 'Basic_Demos_Enroll_Year_2018', 'Basic_Demos_Enroll_Year_2019', 'Basic_Demos_Enroll_Year_2020', 'EHQ_EHQ_Total', 'ColorVision_CV_Score', 'APQ_P_APQ_P_CP', 'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PM', 'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial', 'Sex_F']

=== Fold 1 ===
  Dimensione X_train: (1091, 26)
  Dimensione X_test: (122, 26)
  Dimens

In [11]:
# === FEATURE SELECTION MANUALE PER DATI DEMOGRAFICI ===
# DATI DI TRAIN
from sklearn.feature_selection import SelectKBest, f_classif


# Lista delle features (colonne) che vuoi eliminare
features_da_eliminare = ["Basic_Demos_Study_Site_2", "Basic_Demos_Study_Site_3", "Basic_Demos_Study_Site_4", "MRI_Track_Scan_Location"]

# Salva numero di feature iniziali (INCLUDENDO ID e target)
n_feature_iniziali = train_final_demo.shape[1]

# Elimina le features specificate dal DataFrame
train_final_demo_selected = train_final_demo.drop(columns=features_da_eliminare, errors='ignore')

# Salva numero di feature dopo rimozione manuale
n_feature_post_manuale = train_final_demo_selected.shape[1]


# === MODELLAZIONE CON RANDOM FOREST (300 estimatori) PER ADHD USANDO DATI DEMOGRAFICI ===

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Features e target
X = train_final_demo_selected.drop(columns=['participant_id', 'ADHD_Outcome'])
y = train_final_demo_selected['ADHD_Outcome'].astype(int)

# Cross-validation stratificata su ADHD_Outcome
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Liste per salvare i risultati di ogni fold
acc_scores_adhd = []
f1_scores_adhd = []

# Cross-validation
for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
    print(f"\n=== Fold {fold} ===")

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    print(f"  Dimensione X_train: {X_train.shape}")
    print(f"  Dimensione X_test: {X_test.shape}")
    print(f"  Dimensione y_train: {y_train.shape}")
    print(f"  Dimensione y_test: {y_test.shape}")
    print(f"  Valori unici in y_train: {np.unique(y_train)}")
    print(f"  Valori unici in y_test: {np.unique(y_test)}")

    # === Standardizzazione === (facoltativa per Random Forest, ma la manteniamo per coerenza)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # === Modello Random Forest ===
    rf = RandomForestClassifier(
        n_estimators=300,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    )
    rf.fit(X_train_scaled, y_train)

    y_pred = rf.predict(X_test_scaled)

    print(f"  Predizioni y_pred (prime 5): {y_pred[:5]}")
    print(f"  Valori reali y_test (prime 5): {y_test[:5].values}")

    # Metriche per questo fold
    acc_scores_adhd.append(accuracy_score(y_test, y_pred))
    f1_scores_adhd.append(f1_score(y_test, y_pred))


# === RISULTATI FINALI ===
print("\n=== Media delle metriche su 10 fold ===")
print(f"F1-score medio ADHD_Outcome: {np.mean(f1_scores_adhd):.4f}")
print(f"Deviazione standard F1-score ADHD_Outcome: {np.std(f1_scores_adhd):.4f}")




=== Fold 1 ===
  Dimensione X_train: (1091, 30)
  Dimensione X_test: (122, 30)
  Dimensione y_train: (1091,)
  Dimensione y_test: (122,)
  Valori unici in y_train: [0 1]
  Valori unici in y_test: [0 1]
  Predizioni y_pred (prime 5): [1 0 0 1 0]
  Valori reali y_test (prime 5): [1 0 1 1 1]

=== Fold 2 ===
  Dimensione X_train: (1091, 30)
  Dimensione X_test: (122, 30)
  Dimensione y_train: (1091,)
  Dimensione y_test: (122,)
  Valori unici in y_train: [0 1]
  Valori unici in y_test: [0 1]
  Predizioni y_pred (prime 5): [1 1 1 0 1]
  Valori reali y_test (prime 5): [1 1 0 0 1]

=== Fold 3 ===
  Dimensione X_train: (1091, 30)
  Dimensione X_test: (122, 30)
  Dimensione y_train: (1091,)
  Dimensione y_test: (122,)
  Valori unici in y_train: [0 1]
  Valori unici in y_test: [0 1]
  Predizioni y_pred (prime 5): [1 1 1 0 1]
  Valori reali y_test (prime 5): [1 0 1 0 1]

=== Fold 4 ===
  Dimensione X_train: (1092, 30)
  Dimensione X_test: (121, 30)
  Dimensione y_train: (1092,)
  Dimensione y_te

In [12]:
# === FEATURE SELECTION MANUALE PER DATI DEMOGRAFICI ===
from sklearn.feature_selection import SelectKBest, f_classif
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt

# COPIA ORIGINALE DEL DATASET (se vuoi riutilizzare train_final_demo dopo)
train_final_demo_copy = train_final_demo.copy()

# === AUTOMATICA ===
features_da_eliminare = ["Basic_Demos_Study_Site_2", "Basic_Demos_Study_Site_3", "Basic_Demos_Study_Site_4", "MRI_Track_Scan_Location"]
train_final_demo_auto = train_final_demo_copy.drop(columns=features_da_eliminare, errors='ignore')

colonne_da_aggiungere = ['participant_id','ADHD_Outcome']
colonne_da_aggiungere_presenti = [col for col in colonne_da_aggiungere if col in train_final_demo_auto.columns]

train_numeriche = train_final_demo_auto.select_dtypes(include=['int64', 'float64'])
feature_data = train_numeriche.drop(columns=[col for col in colonne_da_aggiungere_presenti if col in train_numeriche.columns], errors='ignore')

y_kbest = train_final_demo_auto['ADHD_Outcome'].astype(int)

k = 26
selector = SelectKBest(score_func=f_classif, k=k)
X_selected = selector.fit_transform(feature_data, y_kbest)

feature_mask = selector.get_support()
selected_features = feature_data.columns[feature_mask]

train_final_demo_selected_auto = pd.concat(
    [train_final_demo_auto[colonne_da_aggiungere_presenti], feature_data[selected_features]],
    axis=1
)

# === RANDOM FOREST CON FEATURE SELECTION AUTOMATICA ===
X = train_final_demo_selected_auto.drop(columns=['participant_id', 'ADHD_Outcome'])
y = train_final_demo_selected_auto['ADHD_Outcome'].astype(int)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
f1_scores_auto = []

for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    rf = RandomForestClassifier(n_estimators=300, class_weight='balanced', random_state=42, n_jobs=-1)
    rf.fit(X_train_scaled, y_train)
    y_pred = rf.predict(X_test_scaled)

    f1_scores_auto.append(f1_score(y_test, y_pred))


# === MANUALE ===
train_final_demo_manu = train_final_demo_copy.drop(columns=features_da_eliminare, errors='ignore')

X = train_final_demo_manu.drop(columns=['participant_id', 'ADHD_Outcome'])
y = train_final_demo_manu['ADHD_Outcome'].astype(int)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
f1_scores_manu = []

for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    rf = RandomForestClassifier(n_estimators=300, class_weight='balanced', random_state=42, n_jobs=-1)
    rf.fit(X_train_scaled, y_train)
    y_pred = rf.predict(X_test_scaled)

    f1_scores_manu.append(f1_score(y_test, y_pred))

# === RISULTATI FINALI ===
print("\n=== RISULTATI F1-SCORE ===")
print("📊 Feature Selection AUTOMATICA (SelectKBest):")
print(f"   ➤ F1-score medio: {np.mean(f1_scores_auto):.4f}")
print(f"   ➤ Deviazione standard: {np.std(f1_scores_auto):.4f}")

print("\n🛠 Feature Selection MANUALE:")
print(f"   ➤ F1-score medio: {np.mean(f1_scores_manu):.4f}")
print(f"   ➤ Deviazione standard: {np.std(f1_scores_manu):.4f}")


=== RISULTATI F1-SCORE ===
📊 Feature Selection AUTOMATICA (SelectKBest):
   ➤ F1-score medio: 0.8516
   ➤ Deviazione standard: 0.0247

🛠 Feature Selection MANUALE:
   ➤ F1-score medio: 0.8566
   ➤ Deviazione standard: 0.0288


In [13]:
f1_scores_ADHD_manu = pd.DataFrame(f1_scores_manu, columns=["F1_ADHD_score"])
f1_scores_ADHD_manu.to_csv('tesi_4.4_RF_ADHD_manuale.csv', index=False)
print("File 'tesi_4.4_RF_ADHD_manuale.csv' salvato con successo!")
f1_scores_ADHD_manu.head()

f1_scores_ADHD_auto = pd.DataFrame(f1_scores_auto, columns=["F1_ADHD_score"])
f1_scores_ADHD_auto.to_csv('tesi_4.4_RF_ADHD_mista.csv', index=False)
print("File 'tesi_4.4_RF_ADHD_mista.csv' salvato con successo!")
f1_scores_sex_auto.head()


import os
print(os.listdir('.')) # Mostra i file nella directory corrente

File 'tesi_4.4_RF_ADHD_manuale.csv' salvato con successo!
File 'tesi_4.4_RF_ADHD_mista.csv' salvato con successo!
['tesi_4.4_RF_ADHD_mista.csv', 'tesi_4.4_SVC_sex_mista.csv', '__notebook__.ipynb', 'tesi_4.4_SVC_sex_manuale.csv', 'tesi_4.4_RF_ADHD_manuale.csv']
